In [1]:
import requests
from io import StringIO
import time

In [ ]:
import cloudscraper
import os

years = list(range(1991, 2024))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

scraper = cloudscraper.create_scraper()

os.makedirs("mvp", exist_ok=True)

for year in years:
    url = url_start.format(year)
    data = scraper.get(url)
    
    with open(f"mvp/{year}.html", "w", encoding="utf-8") as f:
        f.write(data.text)


Fetching https://www.basketball-reference.com/awards/awards_1991.html...
Fetching https://www.basketball-reference.com/awards/awards_1992.html...
Fetching https://www.basketball-reference.com/awards/awards_1993.html...
Fetching https://www.basketball-reference.com/awards/awards_1994.html...
Fetching https://www.basketball-reference.com/awards/awards_1995.html...
Fetching https://www.basketball-reference.com/awards/awards_1996.html...
Fetching https://www.basketball-reference.com/awards/awards_1997.html...
Fetching https://www.basketball-reference.com/awards/awards_1998.html...
Fetching https://www.basketball-reference.com/awards/awards_1999.html...
Fetching https://www.basketball-reference.com/awards/awards_2000.html...
Fetching https://www.basketball-reference.com/awards/awards_2001.html...
Fetching https://www.basketball-reference.com/awards/awards_2002.html...
Fetching https://www.basketball-reference.com/awards/awards_2003.html...
Fetching https://www.basketball-reference.com/award

In [13]:
from bs4 import BeautifulSoup

In [14]:
with open("mvp/1991.html") as f:
    page = f.read()

soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_="over_header").decompose()

In [15]:
mvp_table = soup.find_all(id="mvp")[0]

In [16]:
import pandas as pd

In [17]:
mvp_1991 = pd.read_html(str(mvp_table))[0]

mvp_1991.head()

/var/folders/68/_7k5q9kn4m18rcshhmjmz3ph0000gn/T/ipykernel_38936/4155021141.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_1991 = pd.read_html(str(mvp_table))[0]


,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Michael Jordan,27,CHI,77,891,960,0.928,82,37.0,31.5,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321
1,2,Magic Johnson,31,LAL,10,497,960,0.518,79,37.1,19.4,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251
2,3,David Robinson,25,SAS,6,476,960,0.496,82,37.7,25.6,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264
3,4,Charles Barkley,27,PHI,2,222,960,0.231,67,37.3,27.6,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258
4,5,Karl Malone,27,UTA,0,142,960,0.148,82,40.3,29.0,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225


In [19]:
dfs = []

for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    #soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(StringIO(str(mvp_table)))[0]
    mvp_df["Year"] = year
    
    dfs.append(mvp_df)

IndexError: list index out of range

In [57]:
mvps = pd.concat(dfs)

In [58]:
mvps.to_csv("mvps.csv")

In [ ]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in years:
    url = player_stats_url.format(year)

    data = requests.get(url)

    with open("player/{}.html".format(year), "w+") as f:
        f.write(data.text)
    
    time.sleep(15)

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [8]:
driver = webdriver.Chrome()

for year in years:
    url = player_stats_url.format(year)

    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(15)

    with open("player/{}.html".format(year), "w+") as f:
        f.write(driver.page_source)

KeyboardInterrupt: 

In [34]:
dfs = []

for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(StringIO(str(player_table)))[0]
    player_df["Year"] = year
    
    dfs.append(player_df)

In [37]:
players = pd.concat(dfs)

In [42]:
players.to_csv("players.csv")

In [46]:
team_standings_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [48]:
for year in years:
    url = team_standings_url.format(year)

    data = requests.get(url)

    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)

    time.sleep(10)

In [53]:
dfs = []

for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    east_table = soup.find_all(id="divs_standings_E")[0]
    east_df = pd.read_html(StringIO(str(east_table)))[0]
    east_df["Year"] = year
    east_df["Team"] = east_df["Eastern Conference"]
    del east_df["Eastern Conference"]

    dfs.append(east_df)

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    west_table = soup.find_all(id="divs_standings_W")[0]
    west_df = pd.read_html(StringIO(str(west_table)))[0]
    west_df["Year"] = year
    west_df["Team"] = west_df["Western Conference"]
    del west_df["Western Conference"]
    
    dfs.append(west_df)

In [54]:
teams = pd.concat(dfs)

teams.head()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets


In [55]:
teams.to_csv("teams.csv")